In [1]:
import json
import pandas as pd
import numpy as np
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from utils import *
import datasets
from inference.inference_utils import extract_predictions

FINETUNE_MODEL_OUTPUTS = '../inference/evalute_outputs/adapters/SciLitLLM/score_only/instruction/step-843/review_evaluation_human_annotation'

aspects = [ 'actionability', 'grounding_specificity','verifiability', 'helpfulness']

gold_data = {}

for aspect in aspects:
    ds = datasets.load_dataset('boda/review_evaluation_human_annotation', name=aspect, split='gold')
    ds = ds.to_pandas()
    gold_data[aspect] = ds

gold-00000-of-00001.parquet:   0%|          | 0.00/557k [00:00<?, ?B/s]

silver-00000-of-00001.parquet:   0%|          | 0.00/580k [00:00<?, ?B/s]

hard-00000-of-00001.parquet:   0%|          | 0.00/188k [00:00<?, ?B/s]

Generating gold split:   0%|          | 0/402 [00:00<?, ? examples/s]

Generating silver split:   0%|          | 0/503 [00:00<?, ? examples/s]

...

silver-00000-of-00001.parquet:   0%|          | 0.00/714k [00:00<?, ?B/s]

hard-00000-of-00001.parquet:   0%|          | 0.00/315k [00:00<?, ?B/s]

Generating gold split:   0%|          | 0/211 [00:00<?, ? examples/s]

Generating silver split:   0%|          | 0/589 [00:00<?, ? examples/s]

Generating hard split:   0%|          | 0/254 [00:00<?, ? examples/s]

In [7]:
final_data = {}

for aspect in aspects:
    finetuning_aspect_outputs_pah = f'{FINETUNE_MODEL_OUTPUTS}/raw_outputs_{aspect}_gold.jsonl'
    finetuning_outputs = []
    basemodel_outputs = []
    with open(finetuning_aspect_outputs_pah, 'r') as f:
        for line in f:
            finetuning_outputs.append(json.loads(line))
    finetuning_outputs = extract_predictions(finetuning_outputs)
    finetuning_outputs = pd.DataFrame(finetuning_outputs)
    ## drop columns that do not have the aspect in the name
    finetuning_outputs = finetuning_outputs.filter(like=aspect, axis=1)

    assert len(finetuning_outputs) == len(gold_data[aspect])

    final_data[aspect] = []
    for i in range(len(gold_data[aspect])):
        if not finetuning_outputs.iloc[i][f'{aspect}_label']:
            continue
        if  finetuning_outputs.iloc[i][f'{aspect}_label'] == 'None':
            continue
        row_data = gold_data[aspect].iloc[i].copy()
        row_data['finetuning_label'] = finetuning_outputs.iloc[i][f'{aspect}_label']
        final_data[aspect].append(row_data)

## convert final_data to pandas dataframe and save them into a excel file
final_data = {k: pd.DataFrame(v) for k,v in final_data.items()}
with pd.ExcelWriter('scilit_gold_labels_predictions.xlsx') as writer:
    for aspect in aspects:
        print(len(final_data[aspect]))
        final_data[aspect].to_excel(writer, sheet_name=aspect, index=False)
        

Parsing error: Expecting value: line 1 column 1 (char 0)
Problematic string: 
295
392
121
162


In [8]:
final_data['verifiability']

,review_point,paper_id,venue,focused_review,batch,verifiability,verifiability_label,verifiability_label_type,id,finetuning_label
0,3. The adopted baseline models are weak. First...,ARR_2022_68_review,ARR_2022,1. Despite the well-motivated problem formulat...,2,"{'annotators': ['6740484e188a64793529ee77', '6...",5,gold,50,5
1,- In section 2.3 the authors use Lample et al....,ACL_2017_71_review,ACL_2017,-The explanation of methods in some paragraphs...,2,"{'annotators': ['6740484e188a64793529ee77', '6...",5,gold,56,4
3,- Two things must be improved in the presentat...,ACL_2017_483_review,ACL_2017,- 071: This formulation of argumentation minin...,2,"{'annotators': ['6740484e188a64793529ee77', '6...",5,gold,64,4
4,3. It would be good to show some empirical evi...,NIPS_2016_9,NIPS_2016,Weakness: The authors do not provide any theor...,3,"{'annotators': ['6740484e188a64793529ee77', '6...",3,gold,79,3
5,2 More analysis and comments are recommended o...,ICLR_2022_1794,ICLR_2022,1 Medical imaging are often obtained in 3D vol...,3,"{'annotators': ['6740484e188a64793529ee77', '6...",5,gold,85,4
...,...,...,...,...,...,...,...,...,...,...
128,2. The meta solvers seem to be centralized con...,NIPS_2017_401,NIPS_2017,Weakness:\n1. There are no collaborative games...,8,"{'annotators': ['boda', '6686ebe474531e4a19756...",5,gold,1108,3
129,"5) Line 285: a bit confused about the phrase ""...",ACL_2017_49_review,ACL_2017,"There are some minor points, listed as follows...",8,"{'annotators': ['boda', '6686ebe474531e4a19756...",X,gold,1117,X
130,- Discrepancy between eq. 9 and Figure 1. From...,NIPS_2019_1338,NIPS_2019,", this paper is a solid submission. The idea i...",8,"{'annotators': ['boda', '6686ebe474531e4a19756...",5,gold,1119,X
131,1. Limited technical novelty. Compare with the...,CsCRTvEZg1,EMNLP_2023,1. Limited technical novelty. Compare with the...,8,"{'annotators': ['boda', '6686ebe474531e4a19756...",5,gold,1121,1
